<a href="https://colab.research.google.com/github/hululuzhu/mario-ai-2023/blob/main/notebooks/mario_ai_e2e_08202023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An end-to-end under-trained Mario AI
- Contact: hululu.zhu@gmail.com
- Last update: 08/20/2023
- Note we used old versions of gym+SB3 for compabitility
- Also note low-memory CPU instance (free tier) is good enough to get started, high-end GPU istances will speed up 2x or so

In [ ]:
# @title Run me to import necessary packages and set path
print("-" * 80, "\nQuiet install. ~5mins, be patient plz.\n" + "-" * 80)
!git clone --quiet https://github.com/hululuzhu/mario-ai-2023.git > /dev/null
!pip install -r mario-ai-2023/requirements.txt > /dev/null 2>&1

import warnings
warnings.filterwarnings('ignore')

import os, sys
sys.path.append(os.path.abspath("/content/mario-ai-2023/lib"))

import game_env
import game_control
from IPython.display import Image as displayImage
from stable_baselines3 import PPO
from game_visual import glance_env, evaluate
import torch
from PIL import Image

In [ ]:
# @title Choose world/stage and initialize game
MY_WORLD = 1 # @param [1,2,3,4,5,6,7,8]
MY_STAGE = 1 # @param [1,2,3,4]
mario_env = game_env.build_single_env(
    game_control.get_env_name(MY_WORLD, MY_STAGE),
    action_list=game_control.SIMPLE_ACTIONS)

In [ ]:
# @title Quick glance of the game with a smart AI

glance_env(mario_env, '/tmp/mario_train_glance_01.gif')
displayImage(open('/tmp/mario_train_glance_01.gif','rb').read())

In [ ]:
# @title Initialize the AI program
mario_ppo = PPO('MlpPolicy', mario_env, tensorboard_log="mario_ai",
                n_steps=128) # Reduce n_steps to be a bit more verbose in logging

In [ ]:
# @title We use TensorBoard to track "Learning" progress, "rollout/ep_rew_mean" is most important
%reload_ext tensorboard
# Note the pattern of logdir, 'mario_ai' matches to mario_ppo definition,
# 'ppo_mlp_1m_N' matches to learn code below
%tensorboard --logdir mario_ai/ppo_mlp_1m_1 --reload_multifile True

In [ ]:
# @title Kick off the training now! A "sufficient" training requires 40 million+....
# TODO: allocate resources and change to larget total_timesteps
mario_ppo.learn(total_timesteps=10_000, tb_log_name='ppo_mlp_1m')

In [ ]:
# @title Evaluate the model after some training
evaluate(mario_ppo, mario_env, '/tmp/mario_train_eval_01.gif', best_of_n=2)
displayImage(open('/tmp/mario_train_eval_01.gif','rb').read())

In [ ]:
# @title Optionally, if you want to save your model and load
# my_ppo.save('/tmp/your_mario_ai_path')
# my_ppo = PPO.load('/tmp/your_mario_ai_path')
# You can resume the training from where you saved!